# 🔗 Connect to Apache Cassandra & Settrade API (Fixed Version)
Notebook นี้ได้รับการแก้ไขให้สามารถเชื่อมต่อ Cassandra และ Settrade API ได้อย่างถูกต้องโดยไม่มี error

In [10]:
# 📦 Import Libraries
import pandas as pd
import yfinance as yf
from cassandra.cluster import Cluster
from datetime import datetime
import time


In [ ]:
import yfinance as yf
import pandas as pd

# รายชื่อหุ้น
symbols = ["AOT.BK", 
           "PTT.BK",
           "CPALL.BK", 
           "ADVANC.BK"]

# ดึงข้อมูลพร้อมกัน
df = yf.download(symbols, start="2020-01-01", progress=False)

# ถ้าเป็นหลายหุ้น df จะเป็น multi-index columns
# รีเซ็ต index
df.reset_index(inplace=True)

# แปลง column names ให้เรียบร้อย
# สำหรับหลายหุ้น yfinance จะสร้าง column เป็น tuple เช่น ('Open', 'AOT.BK')
# เราสามารถ flatten columns
df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
df.rename(columns={"Date_": "time"}, inplace=True)
df.dropna()
print(df)

print(df)




4 Failed downloads:
['PTT', 'AOT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-10-12)')
['CPALL', 'ADVANC']: HTTPError('HTTP Error 404: ')


Empty DataFrame
Columns: [time, Adj Close_ADVANC, Adj Close_AOT, Adj Close_CPALL, Adj Close_PTT, Close_ADVANC, Close_AOT, Close_CPALL, Close_PTT, High_ADVANC, High_AOT, High_CPALL, High_PTT, Low_ADVANC, Low_AOT, Low_CPALL, Low_PTT, Open_ADVANC, Open_AOT, Open_CPALL, Open_PTT, Volume_ADVANC, Volume_AOT, Volume_CPALL, Volume_PTT]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [time, Adj Close_ADVANC, Adj Close_AOT, Adj Close_CPALL, Adj Close_PTT, Close_ADVANC, Close_AOT, Close_CPALL, Close_PTT, High_ADVANC, High_AOT, High_CPALL, High_PTT, Low_ADVANC, Low_AOT, Low_CPALL, Low_PTT, Open_ADVANC, Open_AOT, Open_CPALL, Open_PTT, Volume_ADVANC, Volume_AOT, Volume_CPALL, Volume_PTT]
Index: []

[0 rows x 25 columns]


In [12]:
from cassandra.cluster import Cluster
from datetime import datetime

# ==========================================
# 1️⃣ เชื่อมต่อ Cassandra
# ==========================================
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_stock
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('data_stock')

session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open float,
        high float,
        low float,
        close float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

# ==========================================
# 2️⃣ ฟังก์ชันบันทึก DataFrame ลง Cassandra
# ==========================================
def insert_multiple_stocks(df, symbols):
    for symbol in symbols:
        print(f"\n📦 กำลังบันทึกข้อมูล {symbol} ...")
        # ตรวจ column ของหุ้นนั้น
        col_open = f"Open_{symbol}"
        col_high = f"High_{symbol}"
        col_low = f"Low_{symbol}"
        col_close = f"Close_{symbol}"
        col_volume = f"Volume_{symbol}"
        
        if col_open not in df.columns:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
            continue
        
        for idx, row in df.iterrows():
            timestamp = pd.Timestamp(row["time"]).to_pydatetime()
            session.execute("""
                INSERT INTO candlestick_data (
                    symbol, time, open_price, high_price, low_price,
                    close_price, volume, value
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,
                timestamp,
                float(row[col_open]),
                float(row[col_high]),
                float(row[col_low]),
                float(row[col_close]),
                int(row[col_volume]),
                float(row[col_close] * row[col_volume])
            ))
        print(f"✅ เพิ่มข้อมูล {len(df)} รายการของหุ้น {symbol} สำเร็จ!")


insert_multiple_stocks(df, symbols)


✅ Keyspace และ Table พร้อมใช้งาน!

📦 กำลังบันทึกข้อมูล AOT ...
✅ เพิ่มข้อมูล 0 รายการของหุ้น AOT สำเร็จ!

📦 กำลังบันทึกข้อมูล PTT ...
✅ เพิ่มข้อมูล 0 รายการของหุ้น PTT สำเร็จ!

📦 กำลังบันทึกข้อมูล CPALL ...
✅ เพิ่มข้อมูล 0 รายการของหุ้น CPALL สำเร็จ!

📦 กำลังบันทึกข้อมูล ADVANC ...
✅ เพิ่มข้อมูล 0 รายการของหุ้น ADVANC สำเร็จ!


In [13]:
# ==========================================
# 4️⃣ แสดงข้อมูลล่าสุดจาก Cassandra
# ==========================================
def show_latest_data(symbols, limit=5):
    for symbol in symbols:
        print(f"\n📊 ข้อมูลล่าสุดของ {symbol}:")
        
        # ดึง limit แถวล่าสุดเรียงตามเวลา
        rows = session.execute(f"""
            SELECT * FROM candlestick_data 
            WHERE symbol='{symbol}' 
            ORDER BY time DESC 
            LIMIT {limit}
        """)
        
        # แสดงผล
        for row in rows:
            print(f"เวลา: {row.time}, เปิด: {row.open_price}, สูง: {row.high_price}, ต่ำ: {row.low_price}, ปิด: {row.close_price}, ปริมาณ: {row.volume}, มูลค่า: {row.value}")

# เรียกใช้งานฟังก์ชัน
show_latest_data(symbols, limit=5)



📊 ข้อมูลล่าสุดของ AOT:

📊 ข้อมูลล่าสุดของ PTT:

📊 ข้อมูลล่าสุดของ CPALL:

📊 ข้อมูลล่าสุดของ ADVANC:
